In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
os.listdir('../input/cassava-leaf-disease-classification')

In [ ]:
train_img_path = '../input/cassava-leaf-disease-classification/train_images'
images = os.listdir(train_img_path)

print("Training images:", len(images))

In [ ]:
import json
file = open('../input/cassava-leaf-disease-classification/label_num_to_disease_map.json')

class_map = json.load(file)
print(json.dumps(class_map,indent=1))
lis = json.dumps(class_map)

In [ ]:
type(class_map)

In [ ]:
import pandas as pd

train_file = '../input/cassava-leaf-disease-classification/train.csv'

train = pd.read_csv(train_file)
train.head()

In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

In [ ]:
import cv2

In [ ]:
sample_image = os.path.join(train_img_path, images[5])
img = cv2.imread(sample_image)
img.shape

In [ ]:
torch.cuda.is_available()

In [ ]:
class LeafDataset(torch.utils.data.Dataset):
    def __init__(self, csv_path, images_folder, train, transform = None):
        self.df = pd.read_csv(csv_path)
        self.images_folder = images_folder
        self.transform = transform
        self.train = train
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        filename = self.df["image_id"][index]
        try:
            label = self.df["label"][index]
        except:
            label = ''
        image = PIL.Image.open(os.path.join(self.images_folder, filename))
        if self.transform is not None and self.train:
            image = self.transform['train'](image)
        elif self.transform is not None and not self.train:
            image = self.transform['val'](image)
        return image, label

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
import pandas as pd
import PIL
train_file = '../input/cassava-leaf-disease-classification/train.csv'

train = pd.read_csv(train_file)
train.head()

leaf_data_train = LeafDataset(train_file, train_img_path, True, data_transforms)

In [ ]:
train_size = int(0.8 * len(leaf_data_train))
val_size = len(leaf_data_train) - train_size

In [ ]:
train_size, val_size

In [ ]:
train_dataset, val_dataset = torch.utils.data.random_split(leaf_data_train, [train_size, val_size])

In [ ]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)
validation_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)
dataloaders = {}
dataloaders['train'] = train_loader
dataloaders['val'] = validation_loader
dataset_sizes = {}
dataset_sizes['train'] = len(train_dataset)
dataset_sizes['val'] = len(val_dataset)

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
inputs, classes = next(iter(train_loader))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_map[str(int(x))] for x in classes])

In [ ]:
classes.data

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
len(dataloaders['train'])

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            mini_batch_no = 1
            for inputs, labels in dataloaders[phase]:
                
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                mini_batch_no += 1
                if mini_batch_no%10 == 0:
                    print('{} minibates done'.format(mini_batch_no))
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_map[str(int(preds[j]))]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
# model_ft = models.resnet18(pretrained=True)
model_path_download = '../input/resnet18/resnet18.pth'
model_ft = models.resnet18(pretrained=False)
model_ft.load_state_dict(torch.load(model_path_download))
# model_ft = torch.load(model_path_download)
num_ftrs = model_ft.fc.in_features


model_ft.fc = nn.Linear(num_ftrs, len(class_map))

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
# model_ft = models.resnet18(pretrained=False)
# model_ft.load_state_dict(torch.load(model_path_download))

In [ ]:
# model_ft1 = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
#                        num_epochs=5)
# PATH = os.path.join('../input/cassava-leaf-disease-classification', 'leaf_disease.pt')
torch.save(model_ft1.state_dict(), 'leaf_disease.pt')
# model_ft.load_state_dict(best_model_wts)

In [ ]:
# torch.save(model_ft.state_dict(), 'resnet18.pt')

In [ ]:
os.listdir('/kaggle/working')
# print(os.getcwd())

In [ ]:
visualize_model(model_ft)

In [ ]:
test_img_path = '../input/cassava-leaf-disease-classification/test_images'
test_images = os.listdir(test_img_path)

print("Testing images:", len(test_images))

In [ ]:
model_path_inf = '/kaggle/working/leaf_disease.pt'
model_inf = models.resnet18(pretrained=False)
num_ftrs = model_inf.fc.in_features
model_inf.fc = nn.Linear(num_ftrs, len(class_map))
model_inf.load_state_dict(torch.load(model_path_inf))
# model_inf = models.resnet18(pretrained=True)
# num_ftrs = model_inf.fc.in_features


# model_inf.fc = nn.Linear(num_ftrs, len(class_map))

model_inf = model_inf.to(device)
# PATH = os.path.join('/kaggle/working', 'resnet18.pt')
# model_inf.load_state_dict(torch.load(PATH))

In [ ]:
test_file = "../input/cassava-leaf-disease-classification/sample_submission.csv"
test_df = pd.read_csv("../input/cassava-leaf-disease-classification/sample_submission.csv")
image_path = "../input/cassava-leaf-disease-classification/test_images/"
# fake targets
test_targets = test_df.label.values

In [ ]:
batch_size = 4
test_dataset = LeafDataset(test_file, image_path, False, data_transforms)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)


In [ ]:
predictions=[]

for imgs, _ in test_loader:
    imgs = imgs.to(device)
    with torch.no_grad():
        model_inf = model_inf.to(device)
        outputs = model_inf(imgs)
        _, predicted = torch.max(outputs, dim=1)
        predicted=predicted.to('cpu')
        predictions.append(predicted)

In [ ]:
predictions

In [ ]:
test_df['label'] = np.concatenate(predictions)
test_df.to_csv('submission.csv', index=False)